# HW3 ADM

### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from html.parser import HTMLParser
import codecs
import os
import urllib.request
import urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import requests
from lxml import etree
import csv
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import re

### In this homework, we will see how to download webpages and scrape them. After having done that, we will show how we implemented search engines to match queries with vocabulary extracted (and "cleaned") from the valid webpages.

Please note that, to present a readable jupyter file, we will often show the code applied to one of the three folders. Still, you will be able to find the entire code written for each folder on the python files.

# Part 1

# 1.1

### We first open the html file thanks to the codecs library

In [32]:
link1 =open("C:/Users/chari/HW3ADMGR12/movies1.html", 'r')
link2 =open("C:/Users/chari/HW3ADMGR12/movies2.html", 'r')
link3 =open("C:/Users/chari/HW3ADMGR12/movies3.html", 'r')

In [13]:
class MyHTMLParser(HTMLParser):
    def __init__(self):
        self.d = []
        self.info = [] # This list will be useful to get the info from the infobox of wikipedia
        self.tag = [] # We will use that to know when we have to stop retrieving the paragraphs
        super().__init__()

    def handle_starttag(self, tag, attrs):
        self.tag = []
        self.tag.append(tag)
        return(tag)
        
    def handle_data(self, data):
        if data.startswith('https'):
            self.d.append(data)
        else:
            self.info.append(data)
        return(data)

    def return_data(self):
        return(self.d)
    
    def return_info(self):
        return(self.info)
    
    def return_tag(self):
        if len(self.tag)!= 0:
            return(self.tag[0])
    
# To check if it is an url, we check if it begins with "https"


### We can now simply add all the urls to a list

In [34]:
urls1 = []
urls2 = []
urls3 = []

parser = MyHTMLParser()
for item in list(link1):
    parser.feed(item)
    urls1 += parser.return_data()
parser.close()

parser = MyHTMLParser()
for item in list(link2):
    parser.feed(item)
    urls2 += parser.return_data()
parser.close()

parser = MyHTMLParser()
for item in list(link3):
    parser.feed(item)
    urls3 += parser.return_data()
parser.close()

In [7]:
print(len(urls1),len(urls2),len(urls3))

10000 10000 10000


Instead of creating three different parsers, we decided to create the same one three times, by closing it after the creation of each one of the three lists of URLs.

We can also see that we got the same amount of URLs in each folder.

# 1.2

### We can now move on to the 2nd part of the question

We commented the code because we already retrieved the webpages and we don't need to run it again.

In [17]:
for k in range(1,len(urls1)+1):
    try:
        name = 'C:/Users/chari/HW3ADMGR12/movie1/article'+str(k)+'.html'
        urllib.request.urlretrieve(urls3[k-1], name)
    except urllib.error.HTTPError as err:
        print(err.code)
    except Exception :
        print("error")
        time.sleep(1200)
        urllib.request.urlretrieve(urls3[k-1], name)
    time.sleep(2)

We decided not to put the line "urllib.request.urlretrieve(urls1[k-1], name)" also after the 20 minute break because we haven't faced the exception of Wikipedia not allowing us to download the page. The only exceptio

Still, we saw that two webpages were missing (9998 pages from the folder movies1). We decided to get the indices of the missing ones.

After having tried to add them manually, we saw that the library urllib constantly raised a 404 error, even though the URLs were valid. By looking closer to these two URLs, we can see that wikipedia does not find wikipedia pages on these films. Hence, it is normal that we were not able to retrieve these two files.

# 1.3

### Let us now get the tsv files :

**There is an important thing to note : in order no to take a potential paragraph in the infobox as the 1st section, we added a condition in order not to consider this paragraph. That's why we only consider paragraphs from outside the tables.**

Moreover, we must not consider the cases where the **first** paragraph met in a new section is just a backline, as we want text. 

In [623]:
att = ["Directed by","Produced by","Written by","Starring","Music by","Release date","Running time","Country","Language","Budget"]

for k in range(1,10001):
    
    tot = []
    A = []
    B = []
    name = "C:/Users/chari/HW3ADMGR12/movie1/article_" + str(k) + ".html"
    try:
        file = open(name, encoding='utf8')
        soup = BeautifulSoup(file, 'html.parser')
        
        # We have to check if the page refers to a film or is a disambiguous page. We do not want to save the disambiguous ones.
        disambiguous = soup.find("table", {"id" : "disambigbox"})
        
        if disambiguous is not None :
            continue
            
            
        right_table = soup.find("table", attrs={"class":"infobox vevent"})

        # We get the title of the film

        # We put a condition to ONLY retrieve the title of the film, without the parenthesis containing the word "movie" and the year

        if soup.select_one('#firstHeading').find("i") != None:

            title = soup.select_one('#firstHeading').find("i").text

        else:

            title  = soup.select_one('#firstHeading').text   

        tot.append(title)

        # We get the intro and the plot (1st and 2nd sections)

        intro = ''
        plot = ''


        # We get the intro first

        tag  = soup.select_one('p')

        if tag != None:

            if tag.find_parent('table') == None and tag.text!="\n": # We get the intro if it is not a simple "\n".
                while tag.name == 'p':
                    intro+=tag.text
                    tag = tag.find_next_sibling()


            else: # If the paragraph is in the infobox, we don't append the text but look for the next section

                while tag.text == "\n" or tag.find_parent('table') != None :
                    tag = tag.find_next("p")


                while tag.name == 'p': # While we encounter <p> tags, we add them to the intro. Once a different tag is found, it would mean that we reached the end of the intro
                    intro+=tag.text
                    tag = tag.find_next_sibling()


            # We can now retrieve the second section

            tag = tag.find_next("p") # Now that we left the loop, we are looking for the first p, which would mean we entered the second section

            if tag!= None:

                if tag.find_parent('table') == None and tag.next!="\n" and tag.find_parent('blockquote') == None: # We added a condition on blockquote : we mustn't add quotes as a paragraph ! It's important since some wikipedia pages have an intro in which there is a quote.
                    while tag.name == 'p':                    
                        plot+=tag.text
                        tag = tag.find_next_sibling()


                else:
                    while tag.text == "\n" or tag.find_parent('table') != None or tag.find_parent('blockquote') != None:
                        tag = tag.find_next("p")


                    while tag.name == 'p':
                        plot+=tag.text
                        tag = tag.find_next_sibling()
            else:
                pass

        else :
            pass


        if intro != "":
            intro = intro.replace("\n", "")
            tot.append(intro)
        else :
            tot.append("NA") # If we didn't find an intro, we simply write NA

        if plot != "":
            plot = plot.replace("\n", "")
            tot.append(plot)
        else :
            tot.append("NA")  # If we didn't find a plot, we simply write NA


        # We now want to get the elements from the infobox

        if right_table != None :
            checkcat = 0 # This will enable us to check if a category is linked to some text. It is really important since, sometimes, an image is put instead
            for row in right_table.findAll("tr"):
                cells = row.findAll('td')
                states=row.findAll('th') #To store second column data
                col = MyHTMLParser()
                val = MyHTMLParser()
                if len(states)>=1:
                    col.feed(str(states[0]))
                    a = col.return_info()
                    if len(a)>=1:
                        A.append(a[0])
                    if len(cells)>=1:
                        val.feed(str(cells[0]))
                        b = val.return_info()
                        if len(b)>=1:
                            B.append(b)
                        else:
                            if checkcat != 0: # We want to keep the first element of the infobox, which is the title of the film
                                A.remove(a[0])
                    else:
                        if checkcat != 0: # We want to keep the first element of the infobox, which is the title of the film
                            A.remove(a[0])
                checkcat+=1 


            tot.append(A[0]) # We first append the title of the film

            for i in range(len(att)):
                if att[i] in A[1:]: # We check if we have each piece of information in A and B. If not, we will write "NA".
                    ind = A.index(att[i])
                    if A[ind] == "Release date":
                        if len(B[ind-1])>1:
                            date = ''.join(B[ind-1][2:len(B[ind-1])-1]) # We join all the dates (American date and European date for instance)
                            date = date.replace("\n"," ")
                            tot.append(date)
                        else:
                            tot.append(B[ind-1][0])
                    else:
                        text = " ".join(B[ind-1])
                        text = text.replace("\n", "")
                        tot.append(text)
                else:
                    tot.append("NA")
        else :
            tot = tot + ["NA"]*11 # If we haven't found the infobox, we just append 11 times "NA", as asked on the instructions.


        nametsv = 'C:/Users/chari/HW3ADMGR12/tsv1/urls1output'+str(k)+".tsv"
        with open(nametsv, 'w', newline='',encoding = 'utf-8') as f_output:
            tsv_output = csv.writer(f_output, delimiter='\t')
            tsv_output.writerow(tot)   
    except Exception:
        pass

We looked into the problems that occurred and it resulted on a big amount of exceptions. If we hadn't put these exceptions, we would have got all the tsv files but with FALSE content ! For instance, we could have had many documents with a simple "\n" as an intro ! We considered that we could sacrifice a few exceptions (3 for movies1) for the sake of having valid content in the tsv files.

Please note that we retrieve the release date from index 2 (with the code ''.join(B[ind-1][2:len(B[ind-1])-1])) in order just to keep the (year/month/day) format. BUT, if there are multiple dates, this won't be true anymore. Anyway, this isn't a problem since it will just also write the litteral version of the date.

We decided not to take the pages where we could find the table "disambiguous", which states that the page doesn't refer to a movie but to a set of wikipedia pages with similar names.

The same process was repeated by the other members of the group on their folders.

# 2) Search engine

# Part 2.1

# 2.1.1

## Preprocessing the data

Before getting into the search engine, we had to "clean" the tsv files. First we removed the stopwords.

We also have to keep the URLs of the webpages we downloaded. Let us set the URLs of the movies which didn't have a wikipedia page to None. The following code is done for the folder movies1. The same process has been reproduced for the other folders.

We could have done it in the previous block of code. Still, we thought that it would be more interesting to divide the steps.

In [35]:
valid1 = [] # This list will store the index of all the existant tsv files

for k in range (1,10001):
    name = "C:/Users/chari/HW3ADMGR12/tsv1/urls1output" + str(k) + ".tsv"
    try:
        open(name,'r')
        valid1.append(k)
    except Exception :
        urls1[k-1] = None

In [36]:
valid2 = [] # This list will store the index of all the existant tsv files

for k in range (1,10001):
    name = "C:/Users/chari/HW3ADMGR12/tsv2/output" + str(k) + ".tsv"
    try:
        open(name,'r')
        valid2.append(k)
    except Exception :
        urls2[k-1] = None

In [37]:
valid3 = [] # This list will store the index of all the existant tsv files

for k in range (1,10001):
    name = "C:/Users/chari/HW3ADMGR12/tsv3/urls3output" + str(k) + ".tsv"
    try:
        open(name,'r')
        valid3.append(k)
    except Exception :
        urls3[k-1] = None

We will need to use the links afterwards, especially for the search engines. In order not to run the previous code too many times, we create files to store all the links.

In [9]:
links1 = 'C:/Users/chari/HW3ADMGR12/links1.json'
links2 = 'C:/Users/chari/HW3ADMGR12/links2.json'
links3 = 'C:/Users/chari/HW3ADMGR12/links3.json'
val1 = 'C:/Users/chari/HW3ADMGR12/valid1.json'
val2 = 'C:/Users/chari/HW3ADMGR12/valid2.json'
val3 = 'C:/Users/chari/HW3ADMGR12/valid3.json'

with open(links1, 'w') as f:
    json.dump(urls1, f)
    
with open(links2, 'w') as f:
    json.dump(urls2, f)

with open(links3, 'w') as f:
    json.dump(urls3, f)
    
with open(val1, 'w') as f:
    json.dump(valid1, f)

with open(val2, 'w') as f:
    json.dump(valid2, f)
    
with open(val3, 'w') as f:
    json.dump(valid3, f)

#### We now do not need to run all the code above to get the URLs. We can simply call the documents we created.

In [30]:
links1 = "C:/Users/chari/HW3ADMGR12/links1.json"
links2 = "C:/Users/chari/HW3ADMGR12/links2.json"
links3 = "C:/Users/chari/HW3ADMGR12/links3.json"
val1 = 'C:/Users/chari/HW3ADMGR12/valid1.json'
val2 = 'C:/Users/chari/HW3ADMGR12/valid2.json'
val3 = 'C:/Users/chari/HW3ADMGR12/valid3.json'

with open(links1) as json_file:
    urls1 = json.load(json_file)

with open(links2) as json_file:
    urls2 = json.load(json_file)
    
with open(links3) as json_file:
    urls3 = json.load(json_file)
    
with open(val1) as json_file:
    valid1 = json.load(json_file)

with open(val2) as json_file:
    valid2 = json.load(json_file)
    
with open(val3) as json_file:
    valid3 = json.load(json_file)

Now that we only kept the URLs of the valid webpages, we can start to create the vocabulary. Dictionaries are used there in order to link a word to all the documents in which it can be found.

In [11]:
vocab = []
dicti = {} # This dictionary will enable us to fill the index dictionary (for the Inverted index)
index = {} # This dictionary will contain the "patterns" of words as keys and the name of the documents in which they are

First, we created a function to be able to create the vocabulary and, afterwards, the index. It takes information from each folder (movies1, 2 or 3) and returns the updated version of the dictionary and a dictionary we will use to create the index.

In [12]:
def creationind(folder,valid,tsv,dicti,vocab):
    for k in valid:

        file = open(tsv + str(k) + ".tsv", "r", encoding='utf-8')
        data = list(file)[0]
        data = data.split("\t")
        data = data[1:3]
        data = " ".join(data)
        tokenizer = RegexpTokenizer(r'\w+')

        words = tokenizer.tokenize(data)

        wordsFiltered = []

        stopWords = set(stopwords.words('english'))

        for w in words:
            if w.lower() not in stopWords:  # We used the lower() function because some stopwords can be written with an uppercase in the tsv files.
                wordsFiltered.append(w)

        ps = PorterStemmer()

        for w in wordsFiltered:

            if ps.stem(w) not in vocab:
                vocab.append(ps.stem(w))
                dicti[ps.stem(w)] = ["document"+str(folder)+"_" + str(k)]
            else:
                if "document"+str(folder)+"_" + str(k) not in dicti[ps.stem(w)]:
                    dicti[ps.stem(w)].append("document"+str(folder)+"_" + str(k))
    return(dicti,vocab)


In [10]:
tsv1 = "C:/Users/chari/HW3ADMGR12/tsv1/urls1output"
tsv2 = "C:/Users/chari/HW3ADMGR12/tsv2/output"
tsv3 = "C:/Users/chari/HW3ADMGR12/tsv3/urls3output"


with open(links1) as json_file:
    urls1 = json.load(json_file)

with open(links2) as json_file:
    urls2 = json.load(json_file)

with open(links3) as json_file:
    urls3 = json.load(json_file)

with open(val1) as json_file:
    valid1 = json.load(json_file)

with open(val2) as json_file:
    valid2 = json.load(json_file)

with open(val3) as json_file:
    valid3 = json.load(json_file)
    
folder = 1

allinfos = creationind(folder,valid1,tsv1,dicti,vocab)
dicti = allinfos[0]
vocab = allinfos[1]

folder = 2

allinfos = creationind(folder,valid2,tsv2,dicti,vocab)
dicti = allinfos[0]
vocab = allinfos[1]

folder = 3

allinfos = creationind(folder,valid3,tsv3,dicti,vocab)
dicti = allinfos[0]
vocab = allinfos[1]

**As you can see, we created a function to handle the creation of the vocabulary. It makes the code more readable.**

We now have all the words we could find in the intro and the plot of each of the 30000 movies. We can now create the inverted index. We just go over the list *vocab* and append to a new dictionary, "index", the index of each word and the list of documents in which we can find the word.

In [18]:
voc = {} # We will store the vocabulary into a json file, thanks to a dictionary

    
# Finally, we got the inverted index and the vocabulary.

for k in range (len(vocab)):
    voc[vocab[k]] = k
    index[k] = dicti[vocab[k]]

We can now create the vocabulary file and the index :

In [19]:
vocabulary = "C:/Users/chari/HW3ADMGR12/vocabulary.json"

with open(vocabulary, 'w') as f:
    json.dump(voc, f)


In [20]:
indexn = 'C:/Users/chari/HW3ADMGR12/index.json'

with open(indexn, 'w') as f:
    json.dump(index, f)

### Now that we got the files we need, we can compute the first search engine.

# 2.1.2

In [34]:
vocabulary = "C:/Users/chari/HW3ADMGR12/vocabulary.json"
indexn = 'C:/Users/chari/HW3ADMGR12/index.json'

with open(indexn) as json_file:
    index = json.load(json_file)
    
with open(vocabulary) as json_file:
    voc = json.load(json_file)

#### In order not to run again the code to create the vocabulary and the inverted index, we simply load the files we created. Even though the index is a bit "heavy" (92.9 MB), the code is run fastly. 

It could have got slower if the vocabulary was bigger; if we consider more wikipedia pages.

In [31]:
links1 = "C:/Users/chari/HW3ADMGR12/links1.json"
links2 = "C:/Users/chari/HW3ADMGR12/links2.json"
links3 = "C:/Users/chari/HW3ADMGR12/links3.json"
val1 = 'C:/Users/chari/HW3ADMGR12/valid1.json'
val2 = 'C:/Users/chari/HW3ADMGR12/valid2.json'
val3 = 'C:/Users/chari/HW3ADMGR12/valid3.json'

with open(links1) as json_file:
    urls1 = json.load(json_file)

with open(links2) as json_file:
    urls2 = json.load(json_file)
    
with open(links3) as json_file:
    urls3 = json.load(json_file)
    
with open(val1) as json_file:
    valid1 = json.load(json_file)

with open(val2) as json_file:
    valid2 = json.load(json_file)
    
with open(val3) as json_file:
    valid3 = json.load(json_file)

In [36]:
query = input()

Disney movies 2019


Now that the query is entered, we split it and apply the same steps we applied to the words of the tsv files. In that way, we would pre-process the query to get the most accurate results.

In [39]:
allquery = query.split()


# ---------- We have to apply the same process (stemming,removing stopwords...) on the words of the query :

wordsFiltered = []

allquery = query.split()    


tokenizer = RegexpTokenizer(r'\w+')



stopWords = set(stopwords.words('english'))


for w in allquery:
    if w.lower() not in stopWords: # We used the lower() function because some stopwords can be written with an uppercase in the tsv files.
        wordsFiltered.append(w.lower())

allquery = []
ps = PorterStemmer()
for w in wordsFiltered:
    allquery.append(ps.stem(w))

# -----------------------------------------------------------        

subresult = [] # This list will contain each result of the query, before storing it in the dataframe "result".
d = []

if all(elem in voc.keys() for elem in allquery):
    
    for i in range(len(allquery)):
    
        d.append(index[str(list(voc.keys()).index(allquery[i]))])
       
    alldocs = list(set(d[0]).intersection(*d)) # We get the intersection of all the documents that contain ALL the words of the query.

    
    for k in range(len(alldocs)):
        
        iddoc = int(list(set(d[0]).intersection(*d))[k].split("_")[1]) # This is the id of the document. We can now retrieve the info from the targeted movie
        
        if list(set(d[0]).intersection(*d))[k].split("_")[0] == 'document1' : # If the document in which we find the word has that name (+"_k"), we retrieve the tsv files from the set of movies1
            file = open("C:/Users/chari/HW3ADMGR12/tsv1/urls1output"+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls1[iddoc-1]
            
        elif list(set(d[0]).intersection(*d))[k].split("_")[0] == 'document2':
            file = open("C:/Users/chari/HW3ADMGR12/tsv2/output"+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls2[iddoc-1]
            
            
        else:
            file = open("C:/Users/chari/HW3ADMGR12/tsv3/urls3output"+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls3[iddoc-1]
            
            
            
        data = list(file)[0]
        data = data.split("\t")
        subresult.append([data[0],data[1],url])
        
        
# Let us also create a dataframe to present the result of the query
        
result = pd.DataFrame(subresult,columns = ["Title","Intro","Url"])
result

,Title,Intro,Url
0,The Chronicles of Narnia: Prince Caspian,"""The Chronicles of Narnia: Prince Caspian is a...",https://en.wikipedia.org/wiki/The_Chronicles_o...
1,Frozen,"""Frozen is a 2013 American 3D computer-animate...",https://en.wikipedia.org/wiki/Frozen_(2013_film)
2,The Avengers,Marvel's The Avengers[6] (classified under the...,https://en.wikipedia.org/wiki/The_Avengers_(20...
3,Cars 3,Cars 3 is a 2017 American 3D computer-animated...,https://en.wikipedia.org/wiki/Cars_3
4,Thumbelina,Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...


As you can see, we recovered the names of the documents. After that, we considered each name of the document in the resut of the query. We retrieved the id of the document, so that we could feed the dataframe "result" with the title, intro and url of **each film which is related to all of the words in the query**.

# Part 2.2

### We will get a bit more in detail by calculating the tf-idf of each word, in order to build another search engine.

### Then we're going to calculate similarity for each parameter and at the end calculate a weighted sum over all of them.

#### So at first, we use search engine of part 2.2 to get the results of the text query.

# 2.2.1

In [12]:
with open("vocabulary.json") as json_file:
    all_words = json.load(json_file)
with open("index.json") as json_file:
    indices = json.load(json_file)

We wrote the following code twice. The first one is useful to build the inverted index and the second will be used during each search made. We did not see the point in storing the idf dictionary as the running time is short.

In [3]:
N = 29501 # total number of documents
#compute IDF
idfDict = {}
for word in all_words.keys():
    word_index = all_words[word]
    df = len(indices[str(word_index)])
    idfDict[word] = math.log(N/float(df))

In [20]:
inverted_index = {k: [] for k in indices.keys()}

def update_inverted_index(path, folder):
    for k in range(1, 10001): 
        try:
            file = open(path+str(k)+".tsv","r",encoding='utf-8')
            data = list(file)[0]
            data = data.split("\t")
            data = data[:3]
            data = " ".join(data)
            tokenizer = RegexpTokenizer(r'\w+')
            words = tokenizer.tokenize(data)
            wordsFiltered = []
            stopWords = set(stopwords.words('english'))

            for w in words:
                if w.lower() not in stopWords: # We used the lower() function because some stopwords can be written with an uppercase in the tsv files.
                    wordsFiltered.append(w)

            word_dict = {}
            ps = PorterStemmer() 
            for w in wordsFiltered:
                word_st = ps.stem(w)
                if  word_st not in word_dict:
                    word_dict[word_st] = 1
                else:
                    word_dict[word_st] += 1

            #compute tf and tfidf
            for word, count in word_dict.items():
                tf = count/float(len(wordsFiltered))
                tfidf = tf * idfDict[word]
                word_index = all_words[word]
                #update the inverted index
                inverted_index[str(word_index)].append(('Document' + str(folder) + "_" + str(k), tfidf))
        except:
            pass


update_inverted_index('C:/Users/chari/HW3ADMGR12/tsv1/urls1output', 1)
update_inverted_index('C:/Users/chari/HW3ADMGR12/tsv2/output', 2)
update_inverted_index('C:/Users/chari/HW3ADMGR12/tsv3/urls3output', 3)


In [21]:
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)

Now that we created the inverted index (through a json file), we can focus on the search engine. As we do not want to create the index at each serch, we load it in the following code.

# 2.2.2

In [15]:
inverted_index = 'C:/Users/chari/HW3ADMGR12/inverted_index.json'
all_words = 'C:/Users/chari/HW3ADMGR12/vocabulary.json'
indices = 'C:/Users/chari/HW3ADMGR12/index.json'


with open(inverted_index) as json_file:
    inverted_index = json.load(json_file)

with open(all_words) as json_file:
    all_words = json.load(json_file)
    
with open(indices) as json_file:
    indices = json.load(json_file)

In [16]:
N = 29501 # total number of documents
#compute IDF
idfDict = {}
for word in all_words.keys():
    word_index = all_words[word]
    df = len(indices[str(word_index)])
    idfDict[word] = math.log(N/float(df))

In [24]:
query = input()

Disney movies 2019


In [17]:
#now that we have inverted_index we can handle the queries

allquery = query.split()
# ---------- We have to apply the same process (stemming,removing stopwords...) on the words of the query :
wordsFiltered = []
allquery = query.split()    
tokenizer = RegexpTokenizer(r'\w+')
stopWords = set(stopwords.words('english'))

for w in allquery:
    if w.lower() not in stopWords: # We used the lower() function because some stopwords can be written with an uppercase in the tsv files.
        wordsFiltered.append(w.lower()) 
allquery = []
ps = PorterStemmer()
for w in wordsFiltered:
    allquery.append(ps.stem(w))
    
    
subresult = [] # This list will contain each result of the query
d = []

if all(elem in all_words.keys() for elem in allquery):    
    for w in allquery:   
        d.append(indices[str(all_words[w])]) 
    alldocs = list(set(d[0]).intersection(*d))
    
    for k in range(len(alldocs)):        
        name = alldocs[k].split("_")
        iddoc = int(name[1]) # This is the id of the document. 
        folder = int(name[0][-1])
        #We can now retrieve the info from the targeted movie
        if folder == 1:
            file = open('C:/Users/chari/HW3ADMGR12/tsv1/urls1output'+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls1[iddoc-1]
        elif folder == 2:
            file = open('C:/Users/chari/HW3ADMGR12/tsv2/output'+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls2[iddoc-1]
        elif folder == 3:
            file = open('C:/Users/chari/HW3ADMGR12/tsv3/urls3output'+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls3[iddoc-1]
        data = list(file)[0]
        data = data.split("\t")
        subresult.append([data[0],data[1],url, iddoc, folder])

In [25]:
#first we compute tf-idf for each word in the query
if len(subresult) != 0:
    word_dict = {}
    query_tfidf = [] #vector to store all tf-idf scores
    #compute tf-idf for each word in the query
    for w in allquery: 
        if  w not in word_dict:
            word_dict[w] = 1
        else:
            word_dict[w] += 1
    for w in allquery:
        count = word_dict[w]
        tf = count/float(len(allquery))
        tfidf = tf * idfDict[word]
        query_tfidf += [tfidf]

        
        
#now we need to create the same vector for each result
results = []
for item in subresult:
    word_dict = {}
    tfidf_vector = []
    for w in allquery:
        for elem in inverted_index[str(all_words[w])]:
            if elem[0] == 'Document' + str(item[4]) + '_' + str(item[3]):
                tfidf_vector.append(elem[1])
    #now that we have the tf-idf vector we can calculate similarity
    similarity = cosine_similarity([query_tfidf], [tfidf_vector])[0][0]
    results.append((similarity, item))
k = 10 #how many results are we going to show
results = heapq.nlargest(10, results)
final_results = []
for item in results:
    final_results.append([item[1][0], item[1][1], item[1][2], item[0]])
result = pd.DataFrame(final_results,columns = ["Title","Intro","Url", "Similarity"])
result

,Title,Intro,Url,Similarity
0,Thumbelina,Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...,0.929615
1,The Avengers,Marvel's The Avengers[6] (classified under the...,https://en.wikipedia.org/wiki/The_Avengers_(20...,0.929615
2,Cars 3,Cars 3 is a 2017 American 3D computer-animated...,https://en.wikipedia.org/wiki/Cars_3,0.878929
3,Frozen,"""Frozen is a 2013 American 3D computer-animate...",https://en.wikipedia.org/wiki/Frozen_(2013_film),0.810984
4,The Chronicles of Narnia: Prince Caspian,"""The Chronicles of Narnia: Prince Caspian is a...",https://en.wikipedia.org/wiki/The_Chronicles_o...,0.771815


As you can see, the result respects the instruction of showing k results (10, here) if we have k results out of the query. If not, we simply print all the results we find. Here, as the query *Disney movies 2019* only gives 5 results, we display them all. Also, we can see that the movie webpage that has most similarity with the query is **Thumbelina**.

# Part 3

## We decided to take several additional information into account. After entering the "normal" query (as in part 2.1), the user will be able to specify his request, by entering the release year of the movies, the starring actors/acctresses...

We now want to let the user enter another information that would lead us to rank the results of the query according it.

First, in order not to scroll up, we load the URLs, the vocabulary and the index in the following block of code.

In [10]:
links1 = "C:/Users/chari/HW3ADMGR12/links1.json"
links2 = "C:/Users/chari/HW3ADMGR12/links2.json"
links3 = "C:/Users/chari/HW3ADMGR12/links3.json"

with open(links1) as json_file:
    urls1 = json.load(json_file)

with open(links2) as json_file:
    urls2 = json.load(json_file)
    
with open(links3) as json_file:
    urls3 = json.load(json_file)
    

In [11]:
with open("vocabulary.json") as json_file:
    all_words = json.load(json_file)
with open("index.json") as json_file:
    indices = json.load(json_file)

#### We can now enter the query

In [20]:
#now that we have inverted_index we can handle the queries
query = input()
allquery = query.split()
# ---------- We have to apply the same process (stemming,removing stopwords...) on the words of the query :
wordsFiltered = []
allquery = query.split()    
tokenizer = RegexpTokenizer(r'\w+')
stopWords = set(stopwords.words('english'))

for w in allquery:
    if w.lower() not in stopWords: # We used the lower() function because some stopwords can be written with an uppercase in the tsv files.
        wordsFiltered.append(w.lower()) 
allquery = []
ps = PorterStemmer()
for w in wordsFiltered:
    allquery.append(ps.stem(w))

godfather


In [21]:
subresult = [] # This list will contain each result of the query
d = []

if all(elem in all_words.keys() for elem in allquery):    
    for w in allquery:   
        d.append(indices[str(all_words[w])]) 
    alldocs = list(set(d[0]).intersection(*d))
    
    for k in range(len(alldocs)):        
        name = alldocs[k].split("_")
        iddoc = int(name[1]) # This is the id of the document. 
        folder = int(name[0][-1])
        #We can now retrieve the info from the targeted movie
        if folder == 1:
            file = open('C:/Users/chari/HW3ADMGR12/tsv1/urls1output'+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls1[iddoc-1]
        elif folder == 2:
            file = open('C:/Users/chari/HW3ADMGR12/tsv2/output'+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls2[iddoc-1]
        elif folder == 3:
            file = open('C:/Users/chari/HW3ADMGR12/tsv3/urls3output'+str(iddoc)+".tsv","r",encoding='utf-8')
            url = urls3[iddoc-1]
        data = list(file)[0]
        data = data.split("\t")
        subresult.append([data,url, iddoc, folder])

#### The following code enables the user to choose a category of the data stored in the tsv files.
#### He can then enter the name of an actor, of a musician or also the running time of a movie.

In [22]:
query = {'text': allquery,'name':[], 'director':[], 'producer':[], 'writer':[], 'starring':[],
         'music':[], 'release_date':[], 'runtime':[], 'country':[], 'language':[], 'budget':[]}
number = 1
while number != 0:
    number = int(input("Enter the number of parameter you wish to add:" + 
              "\n1. Film Name\n2. Director\n3. Producer\n4. Writer\n5. " + 
              "Starring\n6. Music\n7. Release Date\n8. Runtime\n9. " + 
               "Country\n10. Language\n11. Budget\nEnter 0 if you're done.\n"))
    if number == 1:
        name = input("Enter Film Name:\n")
        query['name'] += [name]
    elif number == 2:
        director = input("Enter Director's name:\n")
        query['director'] += [director]
    elif number == 3:
        Producer = input("Enter Producer's name:\n")
        query['producer'] += [Producer]
    elif number == 4:
        Writer = input("Enter Writer's name:\n")
        query['writer'] += [Writer]
    elif number == 5:
        Starring = input("Enter Star's name:\n")
        query['starring'] += [Starring]
    elif number == 6:
        Music = input("Enter Musician's name:\n")
        query['music'] += [Music]
    elif number == 7:
        release_date = input("Enter Release Year:\n")
        query['release_date'] += [release_date]
    elif number == 8:
        runtime = input("Enter Runtime in minutes:\n")
        query['runtime'] += [runtime]
    elif number == 9:
        Country = input("Enter Country:\n")
        query['country'] += [Country]
    elif number == 10:
        Language = input("Enter Language:\n")
        query['language'] += [Language]
    elif number == 11:
        Budget = input("Enter Budget:\n")
        query['budget'] += [Budget]

Enter the number of parameter you wish to add:
1. Film Name
2. Director
3. Producer
4. Writer
5. Starring
6. Music
7. Release Date
8. Runtime
9. Country
10. Language
11. Budget
Enter 0 if you're done.
2
Enter Director's name:
coppola
Enter the number of parameter you wish to add:
1. Film Name
2. Director
3. Producer
4. Writer
5. Starring
6. Music
7. Release Date
8. Runtime
9. Country
10. Language
11. Budget
Enter 0 if you're done.
5
Enter Star's name:
brando
Enter the number of parameter you wish to add:
1. Film Name
2. Director
3. Producer
4. Writer
5. Starring
6. Music
7. Release Date
8. Runtime
9. Country
10. Language
11. Budget
Enter 0 if you're done.
6
Enter Musician's name:
rota
Enter the number of parameter you wish to add:
1. Film Name
2. Director
3. Producer
4. Writer
5. Starring
6. Music
7. Release Date
8. Runtime
9. Country
10. Language
11. Budget
Enter 0 if you're done.
0


In [23]:
def scoring_function1(query, info):
    if len(query) == 0 or info == 'NA': 
        similarity = 0.5 # If the user doesn't specify any parameter we consider similarity .5
    else:
        cnt = 0
        for item in query:
            if item.lower() in info.lower():
                cnt += 1
        similarity = cnt/len(query)
    return similarity
    
def scoring_function2(difference, normalization):
    #for quantitative parameters we're going to use a negetive exponential function
    #to calculate the similarity between the query and the actual value
    #the function is similarity = exp (-(difference)/(normalization))
    #when the release year is equal to the query the difference is 0 and similarity is 1
    return math.exp(-(difference/normalization))


In [24]:
def get_results(subresult):
    results = []
    for i in range(len(subresult)):
        similarity = {} # a number between 0 and 1
        similarity['name'] = scoring_function1(query['name'], subresult[i][0][3])
        similarity['director'] = scoring_function1(query['director'], subresult[i][0][4])
        similarity['producer'] = scoring_function1(query['producer'], subresult[i][0][5])
        similarity['writer'] = scoring_function1(query['writer'], subresult[i][0][6])
        similarity['starring'] = scoring_function1(query['starring'], subresult[i][0][7])
        similarity['music'] = scoring_function1(query['music'], subresult[i][0][8])
        similarity['country'] = scoring_function1(query['country'], subresult[i][0][11])
        similarity['language'] = scoring_function1(query['language'], subresult[i][0][12])
        
        if len(query['release_date']) == 0  or subresult[i][0][9] == 'NA': 
            similarity['release_date'] = 0.5
        else: 
            year = re.search(r'\d{4}', subresult[i][0][9]) 
            if year != None:
                year = int(year.group(0))
                similarity['release_date'] = scoring_function2(abs(year - int(query['release_date'][0])), 10)
            else:
                similarity['release_date'] = 0.5

        if len(query['runtime']) == 0  or subresult[i][0][10] == 'NA': 
            similarity['runtime'] = 0.5
        else: 
            runtime = re.search(r'\d+', subresult[i][0][10]) 
            if year != None:
                runtime = int(runtime.group(0))
                similarity['runtime'] = scoring_function2(abs(runtime - int(query['runtime'][0])), 25)
            else:
                similarity['runtime'] = 0.5

    #     print(subresult[i][0][3])
    #     print(similarity)
        final_value = 0
        for key, value in similarity.items():
            if key == 'name':
                final_value += 2*value
            else:
                final_value += value
        results.append((final_value/12, subresult[i]))
    return results


In [25]:
results = get_results(subresult)
k = 10 #how many results are we going to show
results = heapq.nlargest(10, results)
final_results = []
for item in results:
    final_results.append([item[1][0][0], item[1][0][1], item[1][0][2], item[0]])
result = pd.DataFrame(final_results,columns = ["Title","Intro","Url", "Similarity"])
result
    
    

,Title,Intro,Url,Similarity
0,The Godfather,"""The Godfather is a 1972 American crime film d...","""In 1945, at his daughter Connie's wedding to ...",0.583333
1,The Godfather Part II,"""The Godfather Part II is a 1974 American crim...","In 1958, during his son's First Communion part...",0.500000
2,Eadweard (film),Eadweard is a 2015 Canadian drama film written...,A psychological drama centered around world-fa...,0.458333
3,The Rain People,The Rain People is a 1969 film directed by Fra...,Housewife Natalie Ravenna (Shirley Knight) dec...,0.416667
4,The Godfather Part III,The Godfather Part III is a 1990 American crim...,"In 1979, Michael Corleone, approaching 60, is ...",0.416667
5,The Freshman,The Freshman is a 1990 American crime comedy f...,"""Clark Kellogg (Matthew Broderick) leaves his ...",0.416667
6,The Conversation,"""The Conversation is a 1974 American mystery t...","""Harry Caul is a surveillance expert who runs ...",0.416667
7,Z Storm,"Z Storm (Chinese: Z風暴), is a 2014 Hong Kong cr...",This is a story about the biggest financial fr...,0.375000
8,Three Godfathers,Three Godfathers is a 1936 western film direct...,"A week before Christmas, four bandits ride thr...",0.375000
9,The Boat That Rocked,"""The Boat That Rocked (titled Pirate Radio in ...","""In 1966, various pirate radio stations broadc...",0.375000


#### As you can see, the movie that is the nearest to the query is the first *Godfather*. It is logical since all the information entered are linked to this film (Nino Rota made the music, for instance). 

Our third search engine is more precise than the two previous ones. It enables us to give a detailed query to sharpen and improve the accuracy of the result.

# Algorithmic question

The entire code of this part is stored in the file exercise_4.py.

### We'll use a matrix that allow us to count the length of the longest palindromic subsequence. In particular we'll use a dynamic way to do that.

In [2]:
def Palindromic(s):
    l=len(s)
    A=[]
    # A is the matrix that represents the length of the longest palindromic subsequence in the interval [i,j] 
    # for now I initialize it at 0 (i!=j) and 1 (i==j) because at this very fist moment I'll consider only 
    # every single char (and obviously a char is palindromic with itself)
    for i in range(l):
        rowA=[]
        for j in range(l):
            if i==j:
                rowA.append(1)
            else:
                rowA.append(0)
        A.append(rowA)
 
    # Now I have to fill the matrix with the subinterval lenght: note that the longest Pal length is 
    # in position A[0][l-1]: the last element of first row (and for this reason the return is this value)
    for k in range(2, l+1):
        for i in range(l-k+1):
            j = i+k-1
            # I used this print to check every comparison
            #print("s[i],s[j] ", s[i],s[j], " i ",i," j ", j)
            if (s[i] == s[j]):
                A[i][j] = A[i+1][j-1] + 2;
            else:
                A[i][j] = max(A[i][j-1], A[i+1][j])
 
    return A[0][l-1]
 

### Now we have the function and we're able to check the longest  palindromic subsequence length of our string "DATAMININGSAPIENZA"

In [3]:
s1="DATAMININGSAPIENZA"
lp=Palindromic(s1)
print(lp)

7


#### Note: for clarity we specify that the element A[i][j] compares the i-th and the j-th char of the string; so the matrix is composed of the length value of the longest sequence up to that particular comparison (for this reason the final length is in position A[0][l-1])

# Conclusion

In this homework, we have been able to discover, for some of us, how to download webpages and parse them. We had to deal with large amount of data. To be able to keep the information we were interested in, we had to take some decisions and make hypothesis, in particular while creating the TSV files (like being careful that we don't recover paragraphs from the infobox, for instance).

As for the search engine, we saw that storing the whole vocabulary was useful, even though loading it can become slow when the vocabulary becomes too heavy.